In [2]:
import pandas as pd
import numpy as np
import yfinance as yf

In [61]:
def get_stock_prices(tickers, start_date, end_date):
    prices = yf.download(tickers, start=start_date, end=end_date, auto_adjust=False, progress=False)["Adj Close"]
    return prices.dropna()

tickers = ["NVDA", "LLY", "GOOG", "JPM", "^GSPC"]
start_date = "2020-01-01"
end_date = "2025-12-16"

prices = get_stock_prices(tickers, start_date, end_date)
prices

Ticker,GOOG,JPM,LLY,NVDA,^GSPC
Date,,,,,
2020-01-02,67.859543,119.573364,122.998085,5.971078,3257.850098
2020-01-03,67.526535,117.995422,122.588737,5.875504,3234.850098
2020-01-06,69.191551,117.901611,123.044548,5.900144,3246.280029
2020-01-07,69.148376,115.897224,123.277153,5.971577,3237.179932
2020-01-08,69.693291,116.801308,124.393532,5.982776,3253.050049
...,...,...,...,...,...
2025-12-09,317.750000,300.510010,982.219971,184.970001,6840.509766
2025-12-10,321.000000,310.109985,993.640015,183.779999,6886.680176
2025-12-11,313.700012,317.380005,1009.380005,180.929993,6901.000000


In [62]:
def calculate_returns(prices, return_type='simple'):
    prices = prices.sort_index()
    if return_type == 'Simple':
        returns = prices.pct_change()
    elif return_type == 'Log':
        returns = np.log(prices).diff()
    else:
        raise ValueError("Usa 'Simpe' o 'Log' para el Tipo de Cálculo de Retornos")
    return returns.dropna()

In [63]:
returns = calculate_returns(prices, return_type='Log')
returns.head()

Ticker,GOOG,JPM,LLY,NVDA,^GSPC
Date,,,,,
2020-01-03,-0.004919,-0.013284,-0.003334,-0.016136,-0.007085
2020-01-06,0.024358,-0.000795,0.003711,0.004185,0.003527
2020-01-07,-0.000624,-0.017147,0.001889,0.012034,-0.002807
2020-01-08,0.007849,0.007770,0.009015,0.001874,0.004890
2020-01-09,0.010984,0.003645,0.016393,0.010923,0.006633


In [ ]:
rf = 0.03 / 252  # Daily risk-free rate assuming 3% annual

premium = returns - rf

def capm_regression(premium, market_returns):
    import statsmodels.api as sm

    results = {}
    
    for ticker in premium.columns:
        Y = premium[ticker]
        X = sm.add_constant(market_returns)
        model = sm.OLS(Y, X).fit()

        results[ticker] = {
            "Alpha": model.params["const"],
            "Beta": model.params[market_returns.name],
            "t-Alpha": model.tvalues["const"],
            "t-Beta": model.tvalues[market_returns.name],
            "R2": model.rsquared,
            "N": int(model.nobs)
        }

    return pd.DataFrame(results).T

In [65]:
RESULTS = capm_regression(returns.drop(columns="^GSPC"), returns["^GSPC"])
RESULTS["Expected Returns"] = rf * 252 + RESULTS["Beta"] * (returns["^GSPC"].mean() * 252)
RESULTS

,Alpha,Beta,t-Alpha,t-Beta,R2,N,Expected Returns
GOOG,0.000477,1.115850,1.314031,40.751247,0.526416,1496.0,0.135295
JPM,0.000139,1.069064,0.388265,39.685101,0.513182,1496.0,0.130880
LLY,0.001078,0.651154,2.141223,17.146536,0.164431,1496.0,0.091445
NVDA,0.001478,1.777855,2.413367,38.492519,0.497929,1496.0,0.197763
